In [1]:
import os

# get absolute path
base_path = os.getcwd()
print(base_path)

#get data syllable path
data_path = os.path.join(base_path, 'data\\syllable-level')
print(data_path)

d:\IDE_workspace\VSCode\TLCN_Project\ViText2SQL
d:\IDE_workspace\VSCode\TLCN_Project\ViText2SQL\data\syllable-level


In [2]:
train_dataset_path = os.path.join(data_path, 'train.json')
table_dataset_path = os.path.join(data_path, 'tables.json')

In [3]:
import json

train_dataset = json.load(open(train_dataset_path))
table_dataset = json.load(open(table_dataset_path))

In [4]:
def see_query_info(row):
    table_units = []
    for item in row['sql']['from']['table_units']:
        table_units.append(item[1])
    return row['question'], row['query'], table_units

def see_table_schema(schema):
    tables = [(r, []) for r in schema['table_names']]
    for col in schema['column_names'][1:]:
        # add column to coresponding table
        tables[col[0]][1].append(col[1])
    return tables

def find_db_from_query(data):
    db_id = data['db_id']
    for table in table_dataset:
        if table['db_id'] == db_id:
            return table

def find_all_question_and_query_from_table(data):
    db_id = data['db_id']
    question_n_query = []
    for r in train_dataset:
        if r['db_id'] == db_id:
            question_n_query.append(see_query_info)
    return question_n_query

In [10]:
def all_info(data):
    tables = find_db_from_query(data)
    question, query, table_units = see_query_info(data)
    schema = see_table_schema(tables)
    
    core_respond_table = [schema[unit] for unit in table_units]
    return core_respond_table, question, query, schema

In [15]:
id = 10
core_respond_table, query, question, schema = all_info(train_dataset[id])
display(core_respond_table)
print(query)
print(question)

[('tổ chức', ['lục địa', 'trang chủ', 'tên', 'id tổ chức'])]

cho biết trang chủ của ' Đại học Michigan ' .
select trang chủ from tổ chức where tên = "University of Michigan"


TypeError: str.join() takes exactly one argument (2 given)

In [29]:
def table_to_string(tables):
    lst = []
    for table in tables:
        lst.append(table[0] + ' (' + ','.join(table[1]) + ')')
    result = '; '.join(lst)
    return result    

def to_single_dataset(id, data):
    core_respond_table, question, query, _ = all_info(data[id])
    
    instruction = f'Tôi có bảng: {table_to_string(core_respond_table)}. Hãy trả lời câu sau thành câu truy vấn: {question}' 
    return instruction, query

In [30]:
to_single_dataset(2, train_dataset)

("Tôi có bảng: bài báo (tóm tắt,id hội nghị,số lượng trích dẫn,id tạp chí,id bài báo,số lượng trích dẫn,tiêu đề,năm). Hãy trả lời câu sau thành câu truy vấn: cho biết bản tóm tắt của bài báo với tiêu đề là ' làm cho hệ thống cơ sở dữ liệu có thể sử dụng được ' .",
 'select tóm tắt from bài báo where tiêu đề = "Making database systems usable"')

In [36]:
dataset = []
columns = ["instruction", "output"]
for id in range(20):
    data = {}
    instruction, query = to_single_dataset(id, train_dataset)
    data[columns[0]] = instruction
    data[columns[1]] = query
    dataset.append(data)

# json_data = json.dumps(dataset, indent=4, ensure_ascii=False)
# print(json_data)

json.dump(dataset, open('dataset.json', 'w'), ensure_ascii=False, separators=(',', ': '), indent=4)

In [40]:
import random

def generate_dataset(number_of_data, data):
    lst_id = [random.randint(0, len(data)) for _ in range(number_of_data)]
    lst = []
    for id in lst_id:
        lst.append(to_single_dataset(id, data))


[701, 3389, 4660, 5917, 982, 4972, 1093, 6301, 5222, 5357, 5880, 2564, 4989, 1079, 2305, 4910, 4998, 3923, 5003, 326]


In [7]:
all_table = [see_table_schema(table) for table in table_dataset]
len(all_table)

166

In [8]:
ll = {}
for table in all_table:
    ll[len(table)] = ll.get(len(table), 0) + 1

In [69]:
ll

{2: 23,
 11: 6,
 3: 55,
 4: 26,
 15: 2,
 14: 2,
 7: 14,
 5: 10,
 9: 3,
 8: 5,
 12: 3,
 6: 8,
 16: 3,
 26: 1,
 10: 2,
 13: 2,
 17: 1}